In [ ]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *
import numpy as np

In [ ]:
maxh = 0.1
# Finite element space order
order = 2
# Stabilization parameter for ghost-penalty
gamma_stab = 100
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order
r = 0.5 # Radius of the circle
vel = 0.3 # Velocity of the circle
beta0 =1
beta1 = 1
beta2 = 1
beta3 = 1
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
w = vel # maximal normalspeed of the circle
dt = 0.1 # time step size

for t in range(0, 5):
    delta = w*dt*3

    lsetp1 = GridFunction(H1(mesh))
    levelset = CF(sqrt((x-vel*t)**2 + y**2))-r
    InterpolateToP1(levelset,lsetp1)

    lsetp1_extended_plus = GridFunction(H1(mesh))
    levelset_extended_plus = CF(sqrt((x-vel*t)**2 + y**2))-(r+delta)
    InterpolateToP1(levelset_extended_plus, lsetp1_extended_plus)

    lsetp1_extended_minus = GridFunction(H1(mesh))
    levelset_extended_minus = CF(sqrt((x-vel*t)**2 + y**2))-(r-delta)
    InterpolateToP1(levelset_extended_minus, lsetp1_extended_minus)

    ci = CutInfo(mesh, lsetp1)
    hasneg = ci.GetElementsOfType(HASNEG)
    neg = ci.GetElementsOfType(NEG)
    hasif = ci.GetElementsOfType(IF)
    haspos = ci.GetElementsOfType(HASPOS)

    ci1 = CutInfo(mesh, lsetp1_extended_plus)
    hasneg1 = ci1.GetElementsOfType(HASNEG)
    neg1 = ci1.GetElementsOfType(NEG)
    hasif1 = ci1.GetElementsOfType(IF)
    haspos1 = ci1.GetElementsOfType(HASPOS)

    ci2 = CutInfo(mesh, lsetp1_extended_minus)
    hasneg2 = ci2.GetElementsOfType(HASNEG)
    neg2 = ci2.GetElementsOfType(NEG)
    hasif2 = ci2.GetElementsOfType(IF)
    haspos2 = ci2.GetElementsOfType(HASPOS)

    ba_diff = BitArray(len(hasneg1))
    ba_diff[:] = hasneg1
    ba_diff &= ~neg2

    interior_facets = GetFacetsWithNeighborTypes(mesh, a = ba_diff, b=ba_diff)

    
    DrawDC(lsetp1,CF(1), CF(0), mesh, "u")
    Draw(BitArrayCF(ba_diff),mesh,"elements_extended_"+str(dt)) #Drawing the Facettpatch for the ghostpenalty
    #Draw(BitArrayCF(interior_facets), mesh, "interior_facets_"+str(dt)) #Drawing the Facettpatch for the ghostpenalty
    #Draw(lsetp1, mesh, "levelset_"+str(dt)) 
    #Draw(lsetp1,CF(1),CF(0),mesh, "levelset_"+str(t))

In [ ]:
V = VectorH1(mesh, order=order,dgjumps=True)
Q = H1(mesh, order=order-1)
Z = NumberSpace(mesh)

u0 = CF((2*pi*y*cos(pi*(x**2+y**2)), -2*pi*x*cos(pi*(x**2+y**2))))
def jump(f):
    return f - f.Other()
def jumpn(f):
    n = Normalize(grad(lsetp1))
    return Grad(f)*n - Grad(f).Other()*n

In [ ]:
from ngsolve import x, y
from ngsolve import CoefficientFunction, Id

t = 0
dt = 0.5
tend = 5.0
nu = 0.1 # viscosity
cnt = 0

def levelset_function(t):
    return CF(sqrt((x-vel*t)**2 + y**2))-r
while t < tend:
    print ("\rt=", t, end="")
    #print ("t=", t, end="")
    #print ("\rt=", t, end="")
    delta = w*dt*3
    
    lsetp1 = GridFunction(H1(mesh))
    levelset = CF(sqrt((x-vel*t)**2 + y**2)-r)
    #levelset = levelset_function(t)
    InterpolateToP1(levelset,lsetp1)
    #tp1.Set(levelset)

    lsetp1_extended_plus = GridFunction(H1(mesh))
    levelset_extended_plus = CF(sqrt((x-vel*t)**2 + y**2))-(r+delta)
    InterpolateToP1(levelset_extended_plus, lsetp1_extended_plus)
    #lsetp1_extended_plus.Set(levelset_extended_plus)

    lsetp1_extended_minus = GridFunction(H1(mesh))
    levelset_extended_minus = CF(sqrt((x-vel*t)**2 + y**2))-(r-delta)
    InterpolateToP1(levelset_extended_minus, lsetp1_extended_minus)
    #lsetp1_extended_minus.Set(levelset_extended_minus)

    ci = CutInfo(mesh, lsetp1)
    hasneg = ci.GetElementsOfType(HASNEG)
    neg = ci.GetElementsOfType(NEG)
    hasif = ci.GetElementsOfType(IF)
    haspos = ci.GetElementsOfType(HASPOS)

    ci1 = CutInfo(mesh, lsetp1_extended_plus)
    hasneg1 = ci1.GetElementsOfType(HASNEG)
    neg1 = ci1.GetElementsOfType(NEG)
    hasif1 = ci1.GetElementsOfType(IF)
    haspos1 = ci1.GetElementsOfType(HASPOS)

    ci2 = CutInfo(mesh, lsetp1_extended_minus)
    hasneg2 = ci2.GetElementsOfType(HASNEG)
    neg2 = ci2.GetElementsOfType(NEG)
    hasif2 = ci2.GetElementsOfType(IF)
    haspos2 = ci2.GetElementsOfType(HASPOS)

    ba_diff = BitArray(len(hasneg1))
    ba_diff[:] = hasneg1
    ba_diff &= ~neg2

    #interior_facets = GetFacetsWithNeighborTypes(mesh, a = ba_diff, b=ba_diff)
    dx = dCut(lsetp1, NEG, definedonelements=hasneg)
    ds = dCut(lsetp1, IF, definedonelements=hasif)
    dw_interface = dFacetPatch(definedonelements=ba_diff)

    V = Compress(V, GetDofsOfElements(V,hasneg1))
    Q = Compress(Q, GetDofsOfElements(Q,hasneg1))
    X = V*Q*Z
    (u,p,z),(v,q,y) = X.TnT()
    gfu = GridFunction(X)

    h = specialcf.mesh_size
    n = Normalize(grad(lsetp1))

    a = BilinearForm(X)
    stokes = InnerProduct(Grad(u), Grad(v))*dx - div(u)*q*dx - div(v)*p*dx
    stokes += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds #nitzshe stabilization
    stokes += (q*n * u + p*n * v) * ds
    stokes += p*y *dx + q *z*dx
    stokes += 1/nu*beta2*h**-2* InnerProduct(jump(u), jump(v)) * dw_interface #velocity ghost penalty
    stokes += nu*beta2*h**-2* InnerProduct(jump(u), jump(v)) * dw_interface #velocity ghost penalty
    stokes += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface #pressure ghost penalty
    a += stokes
    a += 1/dt * InnerProduct(u, v) * dx # time derivative
    a.Assemble()
    inv = a.mat.Inverse(X.FreeDofs())

    if t == 0:
        gfu.components[0].Set(u0)   

    res = LinearForm(X)
    res += 1/dt*InnerProduct(gfu.components[0], v)*dx
    res.Assemble()
    gfu.vec.data =  inv * res.vec

    t = t + dt; cnt += 1
    
    #print("t=", t, "cnt=", cnt)
    DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")